In [12]:
from IPython.display import Image
import pandas as pd
import numpy as np
import cvxpy as cp


<h1><center>Single filter machine scheduling PMI </center></h1>

#### An example of the scheduling model will be illustrated applied in the tobacco industry.
***
MULFI is a machine that produces five different cigarette filters for exportation (see figure 1). 

![Image of Yaktocat](https://www.hauni.com/fileadmin/produkte/secondary/R_PROTOS_M5e_16_1260n35.jpg)
*<center>Figure 1: Filter production machine; Mulfi<center>*
    
The raw material needed to produce each of the different filters is shown in table 1. The so called "product 0" is not a real product, but a virtual one that relates to the fact that machine always have to start and end production cleaned and with no paper, no mint, and no capsule.

|         Product    | Cotton | Paper | Mint | Capsule |
|:------------------:|:------:|-------|------|---------|
|         0.- Start  | A      | 0     | 0    | 0       |
| 1.- White | A      | A     | 0    | 0       |
|   2.- Red | A      | B     | 0    | 0       |
|  3.- Mint | A      | A     | A    | 0       |
|    4.- Single Caps | A      | B     | 0    | A       |
|      5.- Mega Caps | A      | B     | A    | B       |
<caption>**Table 1: Raw material needed to produce each job**</caption> 

The set-up time and cost associated to each raw material change in the MULFI machine is shown in table 2.

| Material | Change | Requirements | Time (hours) | Cost (usd) |
|:--------:|:------:|:----------------------------------------------------------------------------:|:------------:|:-----------:|
| Paper | 0 to A | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Paper | 0 to B | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Paper | A to B | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Paper | B to A | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Paper | A to 0 | N/A | 0 | \$0 |
| Paper | B to 0 | N/A | 0 | \$0 |
| Mint | 0 to A | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Mint | A to 0 | Exhaustive cleaning (with chemicals), raw material change (two persons) | 1.5 | \$10 |
| Caps | 0 to A | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Caps | 0 to B | Basic cleaning, raw material change (one person) | 0.5 | \$1 |
| Caps | A to 0 | Exhaustive cleaning (with chemicals), raw material change (two persons) | 1.5 | \$5 |
| Caps | B to 0 | Exhaustive cleaning (with chemicals), raw material change (two persons) | 1.5 | \$5 |
| Caps | A to B | Very exhaustive cleaning (with chemicals), raw material change (two persons) | 2 | \$7 |
| Caps | B to A | Very exhaustive cleaning (with chemicals), raw material change (two persons) | 2 | \$7 |
<caption>**Table 2: Set-up time (hours) and cost (usd) associated to each raw material change in mulfi**</caption> 

Changing production from product $i$ to product $j$ can imply changing the cotton, paper, mint and capsule. Summing up the raw material's setup time and cost associated to each possible production change we construct table 3 and 4. 

|     Product     	| 0.- Start 	| 1.- White 	| 2.- Red 	| 3.- Mint 	| 4.- Single Caps 	| 5.- Mega Caps 	|
|:---------------:	|:---------:	|:---------:	|:-------:	|----------	|-----------------	|---------------	|
|       0.- Start 	| 0         	| 0.5       	| 0.5     	| 1        	| 1               	| 1.5           	|
|       1.- White 	| 0         	| 0         	| 0.5     	| 0.5      	| 1               	| 1.5           	|
|         2.- Red 	| 0         	| 0.5       	| 0       	| 1        	| 0.5             	| 1             	|
|        3.- Mint 	| 1.5       	| 1.5       	| 2       	| 0        	| 2.5             	| 1             	|
| 4.- Single Caps 	| 1.5       	| 2         	| 1.5     	| 2.5      	| 0               	| 2.5           	|
|   5.- Mega Caps 	| 3         	| 3.5       	| 3       	| 2        	| 3.5             	| 0             	|
<caption>**Table 3: Set-up time in hours associated to each product change in mulfi**</caption> 

|     Product     	| 0.- Start 	| 1.- White 	| 2.- Red 	| 3.- Mint 	| 4.- Single Caps 	| 5.- Mega Caps 	|
|:---------------:	|:---------:	|:---------:	|:-------:	|----------	|-----------------	|---------------	|
|       0.- Start 	| \$0         	| \$1         	| \$1     	| \$2      	| \$2              	| \$3           	|
|       1.- White 	| \$0         	| \$0         	| \$1     	| \$1      	| \$2              	| \$3           	|
|         2.- Red 	| \$0         	| \$1         	| \$0     	| \$2      	| \$1              	| \$2           	|
|        3.- Mint 	| \$10         	| \$10         	| \$11     	| \$0      	| \$1              	| \$2           	|
| 4.- Single Caps 	| \$5         	| \$6         	| \$5     	| \$7      	| \$0              	| \$8           	|
|   5.- Mega Caps 	| \$15         	| \$16         	| \$15     	| \$6      	| \$17             	| \$0           	|
<caption>**Table 4: Set-up cost in usd associated to each product change in mulfi**</caption> 




|     Product     	|     Job 0 	|     Job 1     |   Job 2   |     Job 3     |     Job 4     |    Job 5   	|
|:---------------:  |:-------------:|:-------------:|:---------:|:-------------:|:-------------:|:-------------:|
|  Production Time 	| 5hrs         	| 6hrs        	| 6hrs     	| 8hrs      	| 12hrs         | 18hrs         |
|      Due Date 	| 30hrs        	| 50hrs         | 80hrs     | 90hrs      	| 10hrs         | 130hrs        |
| Tardiness Penalty | \$400         | \$500         | \$600     | \$600      	| \$800         | \$1000        |
|  Quantity Ordered | 1         	| 4         	| 6     	| 2      	    | 1             | 1           	|









Let’s suppose that Phillip Morris Switzerland places some job orders with different specification (see chart 5) and our job as production planning department would be to find the optimal production schedule in order to minimize the cost.<br/>
**Note:** Each batch has 500,000 units.

In order to do so we can apply the tools learned in the Optimization Method for Business and Machine Learning with Python, and develop an optimization algorithm for job scheduling using Mixed Integer Programming.

The algorithm looks as follow and please note that machine should always start and end with no rawm material (virtual node job 0).

## Mathematical Model
**Objective**

($Z_{min}$)= $\sum_{i=1}^{n}\sum_{j=1}^{n} \sigma_{ij}X_{ij}$+ $\sum_{j=1}^{n}T_jP_j$

Constraints

**1.-** $\sum_{i=1}^{n}X_{ij}=1$                                     
$(j=1,2,\ldots,n ,i\neq j)$

**2.-** $\sum_{j=1}^{n}X_{ij}=1$                                    
$(i=1,2,\ldots,n ,i\neq j)$

**3.-** $C_i-C_j+(M+S_{ij}+(Q_jR_j))X_{ij}+(M-S_{ij}-(Q_iR_i))X_{ij}\le M$
$(i=1,2,\ldots,n ,j=1,2,\ldots,n ,i\neq j)$

**4.-** $(S_{1j}+(Q_jR_j))X_{1j} \le C_j$
$(j=1,2,\ldots,n)$

**5.-** $C_j-D_j \le T_j$
$(j=1,2,\ldots,n)$

**6.-** $X_i,X_j \in (0,1)$

**7.-** $T_j,C_j \ge 0$

Notation & Parameter

$n$: Number of job i,j

$D_j$: Due date

$R_j$: Run time

$P_j$: Tardiness penalty

$Q_j$: Quantity ordered

$S_{ij}$: Setup time

$\sigma _{ij}$: Setup cost


In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
chart1 = pd.read_csv("/Users/Mario Alejandro/PycharmProjects/untitled/DHBW/setup_time.csv")
chart2 = pd.read_csv("/Users/Mario Alejandro/PycharmProjects/untitled/DHBW/setup_cost.csv")


b=chart2.to_numpy()[0:,1:]                         #Set up costs
s=chart1.to_numpy()[4:,1:]                         #Set up times
R=chart1.to_numpy()[0:1,1:]                        #Run time in minutes
D=chart1.to_numpy()[1:2,1:]                        #Due Date in minutes
P=chart1.to_numpy()[2:3,1:]                        #Tardiness Penalty in hours
Q=chart1.to_numpy()[3:4,1:]                        #Number of ordered batches



R=R[0]
D=D[0]
P=P[0]                                              #Tardiness penalty in minutes
Q=Q[0]
chart1
chart2

FileNotFoundError: [Errno 2] File b'/Users/Mario Alejandro/PycharmProjects/untitled/DHBW/setup_time.csv' does not exist: b'/Users/Mario Alejandro/PycharmProjects/untitled/DHBW/setup_time.csv'

In [30]:

N=6                                                 #Number of jobs
M=2000

T=cp.Variable(N, nonneg= True)
x=cp.Variable(shape=(N,N),boolean=True)
c=cp.Variable(N, nonneg=True)                     #COMPLETION TIMES



constraints=[cp.sum(x,axis=0)==1, cp.sum(x,axis=1)==1]


for j in range (N):
    constraints += [(s[0][j]+(Q[j]*R[j]))* x[0][j]<=c[j],c[j]-D[j] <= T[j]]

for i in range (1,N):
    for j in range (N):
        if i != j:
            constraints += [c[i] - c[j] + ((M + (Q[j]*R[j])+s[i][j])*x[i][j]) + ((M-(Q[i]*R[i])-s[j][i])*x[j][i])<=M]
        else:
            constraints += [x[i][j]==0]

obj=cp.Minimize(cp.sum(T*P)/60 + cp.sum(b*x))
problem=cp.Problem(obj,constraints)
problem.solve(solver=cp.ECOS_BB)
print('Optimum = ', round(problem.value), ' at ')
print(np.round(x.value,1))


Optimum =  550.0  at 
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


array([  0.        ,  56.99984726, 115.49979574,  31.49984816,
        13.00000093,  76.49980816])